# <center> **Requesting an external API**

A partir d'une liste de titres de films nous allons requêter l'API publique [https://www.omdbapi.com](https://www.omdbapi.com)

Nous enregistrerons les données non-structurées (résumé et affiche du film) dans une base NoSQL (MongoDB)

In [226]:
%reset

## **Imports**

In [2]:
import math
import copy
import re
import json
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from unidecode import unidecode

# MongoDB / Pymongo
import pymongo
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pprint

pd.set_option('display.max_rows', 10)
tqdm.pandas()

api_key = "b8dd5759"

In [ ]:
url = f"https://www.omdbapi.com/?apikey={api_key}&t=god+save+the+tuche"
r = requests.get(url)
if r.status_code != 200:
    print("Error:", url)

json.loads(r.text)

{'Title': 'God Save the Tuche',
 'Year': '2025',
 'Rated': 'N/A',
 'Released': '05 Feb 2025',
 'Runtime': '95 min',
 'Genre': 'Comedy',
 'Director': 'Jean-Paul Rouve',
 'Writer': 'Nessim Chikhaoui, Julien Hervé, Philippe Mechelen',
 'Actors': 'Jean-Paul Rouve, Isabelle Nanty, Claire Nadeau',
 'Plot': 'N/A',
 'Language': 'French',
 'Country': 'France',
 'Awards': 'N/A',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMGZlY2RhNzUtZmExYy00NzAyLWFkZTctMjA5NmU3N2UyMzRhXkEyXkFqcGc@._V1_SX300.jpg',
 'Ratings': [],
 'Metascore': 'N/A',
 'imdbRating': 'N/A',
 'imdbVotes': '94',
 'imdbID': 'tt33613785',
 'Type': 'movie',
 'DVD': 'N/A',
 'BoxOffice': 'N/A',
 'Production': 'N/A',
 'Website': 'N/A',
 'Response': 'True'}

## **Reading the data**


In [ ]:
df_movies = pd.read_csv('csv/movies_year_2022_to_2025.csv', delimiter = ',', usecols=['title', 'original_title', 'summary', 'url_thumbnail'])
print("Nb movies :", df_movies.shape[0])
df_movies

Nb movies : 750


,title,original_title,summary,url_thumbnail
0,Un autre monde,Un autre monde,"Un cadre d'entreprise, sa femme, sa famille, a...",https://fr.web.img2.acsta.net/c_310_420/pictur...
1,Mourir peut attendre,No Time To Die,"Dans MOURIR PEUT ATTENDRE, Bond a quitté les s...",https://fr.web.img2.acsta.net/c_310_420/pictur...
2,S.O.S. Fantômes : L'Héritage,Ghostbusters: Afterlife,Une mère célibataire et ses deux enfants s'ins...,https://fr.web.img6.acsta.net/c_310_420/pictur...
3,Cruella,Cruella,"Londres, années 70, en plein mouvement punk ro...",https://fr.web.img6.acsta.net/c_310_420/pictur...
4,Dune,Dune,"L'histoire de Paul Atreides, jeune homme aussi...",https://fr.web.img5.acsta.net/c_310_420/pictur...
...,...,...,...,...
745,La Belle et le Clochard,Lady And The Tramp,"Dans une petite ville de Nouvelle-Angleterre, ...",https://fr.web.img6.acsta.net/c_310_420/pictur...
746,Le Colocataire,Un Rubio,Juan doit vite trouver un colocataire après le...,https://fr.web.img4.acsta.net/c_310_420/pictur...
747,Operation Brothers,The Red Sea Diving Resort,"Début des années 1980, le Mossad dirige une op...",https://fr.web.img2.acsta.net/c_310_420/pictur...
748,Victor et Célia,Victor et Célia,"Victor, coiffeur, la trentaine, travaille dans...",https://fr.web.img6.acsta.net/c_310_420/pictur...


In [4]:
def format_string(st):
    ''' format string 
        from "title of the movie" 
        to title+of+the+movie

        Arg: st string to be converted.
    '''
    res = ''
    for c in st:
        if c.isdigit() or c.isalpha() or c.isspace():
            res += unidecode(c)
        else:
            res += ' '
    return '+'.join([word for word in res.split() if len(word) > 1])

def request_omdb_from_title(title):
    ''' Request the omdb API
    
        return a json dictionary with the information about the movie.

        Arg:
         - title: string with title of the movie we want the infos about.
    '''
    url = f"https://www.omdbapi.com/?apikey={api_key}&t={format_string(title)}"
    r = requests.get(url)
    if r.status_code != 200:
        print(f"ERROR {title}, Response Code: {r.status_code}")
        print("Request:", url)
        return {'Response': 'False'}
    return json.loads(r.text)

def get_plot_and_thumbail_from_omdb(title):
    ''' return movie plot and thumbail through an API request.

        return: 
          - plot:      string containing the plot of the movie,
          - thumbnail: string containing the url of the thumbnail.

        Arg: title: string with the title of the movie.
    '''
    plot, thumbnail = '', ''
    res_dict = request_omdb_from_title(title)
    lst_keys = res_dict.keys()
    assert 'Response' in res_dict
    if res_dict['Response'] == 'True':
        # print('res_dict', res_dict)
        assert 'Plot' in lst_keys and 'Poster' in lst_keys
        if res_dict['Plot'] != 'N/A':
            plot = res_dict['Plot']
        if res_dict['Poster'] != '' and res_dict['Poster'] != 'N/A':
            thumbnail = res_dict['Poster']
    return plot + "AND" + thumbnail

## **Get the plot and the thumbnail from the omdb API**

In [5]:
df_movies['temp']      = df_movies['original_title'].apply(get_plot_and_thumbail_from_omdb)
df_movies['plot']      = df_movies['temp'].apply(lambda x : x.split('AND')[0])
df_movies['thumbnail'] = df_movies['temp'].apply(lambda x : x.split('AND')[1])
df_movies['plot']          = np.where(df_movies['plot'] != '', df_movies['plot'], df_movies['summary'])
df_movies['url_thumbnail'] = np.where(df_movies['thumbnail'] != '', df_movies['thumbnail'], df_movies['url_thumbnail'])
df_movies = df_movies[['title', 'original_title', 'plot', 'url_thumbnail']]

In [6]:
df_movies

,title,original_title,plot,url_thumbnail
0,Un autre monde,Un autre monde,"Un Autre Monde, is a short documentary/portrai...",https://m.media-amazon.com/images/M/MV5BNmQyMD...
1,Mourir peut attendre,No Time To Die,James Bond has left active service. His peace ...,https://m.media-amazon.com/images/M/MV5BZGZiOG...
2,S.O.S. Fantômes : L'Héritage,Ghostbusters: Afterlife,When a single mom and her two kids arrive in a...,https://m.media-amazon.com/images/M/MV5BMDJmZm...
3,Cruella,Cruella,"During the 1970s London punk rock revolution, ...",https://m.media-amazon.com/images/M/MV5BMWY5Mj...
4,Dune,Dune,A Duke's son leads desert warriors against the...,https://m.media-amazon.com/images/M/MV5BMGJlMG...
...,...,...,...,...
745,La Belle et le Clochard,Lady And The Tramp,The romantic tale of a sheltered uptown Cocker...,https://m.media-amazon.com/images/M/MV5BMjA1Nm...
746,Le Colocataire,Un Rubio,Juan doit vite trouver un colocataire après le...,https://fr.web.img4.acsta.net/c_310_420/pictur...
747,Operation Brothers,The Red Sea Diving Resort,Israel's Mossad agents attempt to rescue Ethio...,https://m.media-amazon.com/images/M/MV5BODk5Zm...
748,Victor et Célia,Victor et Célia,"Victor, coiffeur, la trentaine, travaille dans...",https://fr.web.img6.acsta.net/c_310_420/pictur...


## **Store the data in a NoSQL database**

In [7]:
# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Create database "allocine" (or selects it if already exists)
mydb = client["allocine"]
# client.drop_database("movies")

# Create a collection "movies" (table in SQL)
col_movies = mydb["movies"]
# col_movies.drop()

In [8]:
# Insertion of movie plots in MongoDB database
col_movies.insert_many(df_movies.to_dict(orient='records')) # TO DO ONLY ONCE
# col_movies.drop()

print(client.list_database_names())
print(mydb.list_collection_names())

# for doc in list(col_movies.find().limit(5)):
#     pprint.pprint(doc)

print("Nb documents:", col_movies.count_documents({}))

['Rennes', 'Rennes2', 'admin', 'allocine', 'config', 'local', 'mydatabase', 'test_queries']
['movies']
Nb documents: 8049


## **Checking what is in the Mongo DB**

In [ ]:
names = ['csv/movies_year_1960_to_1970.csv',
         'csv/movies_year_1970_to_1980.csv',
         'csv/movies_year_1980_to_1990.csv',
         'csv/movies_year_1990_to_1995.csv',
         'csv/movies_year_1995_to_2000.csv',
         'csv/movies_year_2000_to_2003.csv',
         'csv/movies_year_2003_to_2006.csv',
         'csv/movies_year_2006_to_2010.csv',
         'csv/movies_year_2010_to_2015.csv',
         'csv/movies_year_2015_to_2019.csv',
         'csv/movies_year_2019_to_2022.csv',
         'csv/movies_year_2022_to_2025.csv']

lst_df = [pd.read_csv(name, delimiter = ',', usecols=['title', 'original_title', 'summary', 'url_thumbnail'])
          for name in names]

for df in lst_df:
    print(df.shape[0])

df_csv = pd.concat(lst_df)
print("Nb total de films en base SQL", df_csv.shape[0])
# df_csv.to_csv('csv/mongoDB_all_years.csv', sep=',', index = False)

# Reading data fmo Mongo DB
client = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = client["allocine"]
col_movies = mydb["movies"]
print("Nb documents MongoDB:", col_movies.count_documents({}))

# 1960 to 2010 stored in Mongo DB
518 + 678 + 869 + 585 + 807 + 729 + 980 + 600 + 750 + 800 + 750 + 750

518
678
869
585
807
729
980
600
750
800
750
750
Nb total de films en base SQL 8816
Nb documents MongoDB: 8049


8066

In [ ]:
# Store data in csv file
lst = list(col_movies.find())
df_mongo = pd.DataFrame(lst, columns = ['_id', 'title', 'original_title', 'plot', 'url_thumbnail'])
print(df_mongo.shape[0])
df_mongo.to_csv('csv/mongoDB_1960_to_2025.csv', sep=',', index = False)

8049


**Since we added twice the same 807 movies, we have to remove them from MongoDB**

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = client["allocine"]
col_movies = mydb["movies"]

print("Nb documents:", col_movies.count_documents({}))
col_movies.delete_many({'original_title' : {'$in' : df_movies['original_title'].values.tolist()} })
print("Nb documents:", col_movies.count_documents({}))

Nb documents: 4264
Nb documents: 2639


**Looks like more documents have been deleted**<br>
So we will drop the whole data base and use the csv 'mongoDB.csv' to restore the previous database and re-add the movies needed.

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
client.drop_database('allocine')

# Recreate DB
client = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = client["allocine"]
col_movies = mydb["movies"]

## **Query MongoDB Using PyMongo**

### **We create another DB**

In [184]:
client2 = pymongo.MongoClient("mongodb://localhost:27017/")
mydb2 = client2["test_queries"]
# client.drop_database("test_queries")

# Create a collection "movies" (table in SQL)
col_fruits = mydb2["fruits"]
# col_fruits.drop()

col_fruits.insert_many([
        # { "_id": 1, "name": "apples", "qty": 5, "rating": 3, "color": "red", "type": ["fuji", "honeycrisp"] },
        # { "_id": 2, "name": "bananas", "qty": 7, "rating": 4, "color": "yellow", "type": ["cavendish"] },
        { "_id": 3, "name": "oranges", "qty": 6, "rating": 2, "type": ["naval", "mandarin"] },
        { "_id": 4, "name": "pineapple", "qty": 3, "rating": 5, "color": "yellow" },
        { "_id": 5, "name": "test", "qty": 10, "rating": 3, "color": "green", "type": ["fuji", "typeXXX"] },
    ])
# col_fruits.drop()

print(client2.list_database_names())
print(mydb2.list_collection_names())

['Rennes', 'Rennes2', 'admin', 'allocine', 'config', 'local', 'mydatabase', 'test_queries']
['fruits']


**Sources**<br>
https://www.mongodb.com/docs/languages/python/pymongo-driver/current/read/specify-a-query/<br>
https://www.w3resource.com/mongodb/introduction-mongodb.php<br>
https://www.mongodbtutorial.org/mongodb-crud/mongodb-findone/<br>
https://geekflare.com/fr/mongodb-queries-examples/<br>


**Exact Match**

In [175]:
results = col_movies.find({ "title": "Eyes Wide Shut" })
results.to_list()

[{'_id': ObjectId('67b1ae3a1e1d959e842460ed'),
  'title': 'Eyes Wide Shut',
  'original_title': 'Eyes Wide Shut',
  'plot': "A Manhattan doctor embarks on a bizarre, night-long odyssey after his wife's admission of unfulfilled longing.",
  'url_thumbnail': 'https://m.media-amazon.com/images/M/MV5BZTQ0MmM5MDAtYmYyZS00MzlmLTlhZTAtZDJlZWY5ZTZkZjZmXkEyXkFqcGc@._V1_SX300.jpg'},
 {'_id': ObjectId('67b1ae4a1e1d959e84246414'),
  'title': 'Eyes Wide Shut',
  'original_title': 'Eyes Wide Shut',
  'plot': "A Manhattan doctor embarks on a bizarre, night-long odyssey after his wife's admission of unfulfilled longing.",
  'url_thumbnail': 'https://m.media-amazon.com/images/M/MV5BZTQ0MmM5MDAtYmYyZS00MzlmLTlhZTAtZDJlZWY5ZTZkZjZmXkEyXkFqcGc@._V1_SX300.jpg'}]

In [144]:
results = col_fruits.find({ "color": "yellow" })
results.to_list()

[{'_id': 2,
  'name': 'bananas',
  'qty': 7,
  'rating': 4,
  'color': 'yellow',
  'type': ['cavendish']},
 {'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}]

**Partial Match** ???

Avec des index

https://www.mongodb.com/docs/manual/reference/operator/query/text/

In [62]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = client["allocine"]

col_movies = mydb["movies"]
results = col_movies.find({ "title": "The Brutalist" })
results.to_list()

[]

In [61]:
col_movies.delete_one({'title' : "The Brutalist"})

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

**Comparison operators**

list of operators: https://www.mongodb.com/docs/manual/reference/operator/query-comparison/

In [143]:
results = col_fruits.find({ "rating": { "$gt" : 2 }})
for f in results:
    print(f) 

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}


**Logical operators**

https://www.mongodb.com/docs/manual/reference/operator/query-logical/

In [147]:
results = col_fruits.find({ 
    "$or": [
        { "qty": { "$gt": 5 }},
        { "color": "yellow" }
    ]
})
for f in results:
    print(f)

{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}


**Arrays operators**

https://www.mongodb.com/docs/manual/reference/operator/query-array/

In [158]:
for f in col_fruits.find({}):
    print(f)

print('----')
results = col_fruits.find({
    "type" : { "$size": 2 }
})

for f in results:
    print(f)

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}
----
{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}


In [159]:
results = col_fruits.find({
    "type" : { "$all": ['fuji'] }
})

for f in results:
    print(f)

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}


**Element Operator**

to check if a field exists

In [160]:
results = col_fruits.find( { "color" : { "$exists": "true" }} )
for f in results:
    print(f)

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}


**Evaluation operators**

https://www.mongodb.com/docs/manual/reference/operator/query-evaluation/

In [ ]:
results = col_fruits.find({ "name" : { "$regex" : "p{2,}" }} ) ## At least 2 consecutives p
for f in results:
    print(f)

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}


**Delete data**

In [ ]:
# Delete_one
for f in col_fruits.find({}):
    print(f)

col_fruits.delete_one({'_id' : 4})

for f in col_fruits.find({}):
    print(f)

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}
{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}


In [ ]:
# Delete_many
for f in col_fruits.find({}):
    print(f)

col_fruits.delete_many({'_id' : {'$gte' : 3} })

for f in col_fruits.find({}):
    print(f)

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}
{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}


In [186]:
# Delete_many
for f in col_fruits.find({}):
    print(f)

col_fruits.delete_many({'_id' : {'$in' : [1, 3, 5]} })

for f in col_fruits.find({}):
    print(f)

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}
